In [ ]:
!pip install tensorflow
!pip install scikit-learn
!pip install torch torchvision
!pip install torch-geometric
!pip install optuna
!pip install scikit-learn
!pip install torch torchvision torchaudio
!pip install torch-geometric
!pip install timm
# # or for huggingface transformers if you'd like to use that:
!pip install transformers
!pip install scikit-learn
!pip install matplotlib opencv-python
!pip install tensorflow
!pip install keras-tuner

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sivm205/soybean-diseased-leaf-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/soybean-diseased-leaf-dataset


In [7]:
import tensorflow as tf
import tensorflow.keras.layers as layers
import numpy as np
import networkx as nx
import scipy.sparse as sp
import os
import cv2
from tensorflow.keras.applications import MobileNetV2
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import optuna
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Dataset Directory
dataset_dir = "/kaggle/input/soybean-diseased-leaf-dataset"

# Load dataset images and labels
def load_dataset(dataset_dir, img_size=(224, 224)):
    images = []
    labels = []
    class_names = sorted(os.listdir(dataset_dir))
    class_dict = {class_name: idx for idx, class_name in enumerate(class_names)}
    
    for class_name in class_names:
        class_path = os.path.join(dataset_dir, class_name)
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            img = cv2.imread(img_path)
            img = cv2.resize(img, img_size)
            img = img / 255.0  # Normalize image
            images.append(img)
            labels.append(class_dict[class_name])
    
    return np.array(images), np.array(labels)

# Load actual dataset
images, labels = load_dataset(dataset_dir)

# Construct graph using cosine similarity of image features
def create_graph(image_features, threshold=0.8):
    similarity_matrix = cosine_similarity(image_features)
    adj_matrix = (similarity_matrix > threshold).astype(int)
    return sp.coo_matrix(adj_matrix)

# Graph Convolutional Network (GCN) Layer
def create_gcn(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    x = layers.Dense(64, activation='relu')(inputs)
    x = layers.Dense(32, activation='relu')(x)
    outputs = layers.Dense(128, activation='relu')(x)
    return tf.keras.Model(inputs, outputs)

# Load MobileNetV2 as feature extractor
def create_cnn(input_shape):
    base_model = MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')
    base_model.trainable = False
    model = tf.keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(10, activation='softmax')
    ])
    return model

# Sequential GCN-CNN Model
def create_sequential_gnn_cnn(input_shape, num_nodes):
    gcn_model = create_gcn((128,))
    cnn_model = create_cnn((224, 224, 3))
    
    gcn_input = tf.keras.Input(shape=(128,))
    gcn_output = gcn_model(gcn_input)
    
    cnn_input = tf.keras.Input(shape=(224, 224, 3))
    cnn_output = cnn_model(cnn_input)
    
    model = tf.keras.Model(inputs=[gcn_input, cnn_input], outputs=cnn_output)
    return model

# Extract features and create graph
node_features = np.random.rand(len(images), 128)  # Placeholder node features
adj_matrix = create_graph(node_features)

# Train-Test Split
train_data, test_data, train_labels, test_labels, train_node_features, test_node_features = train_test_split(images, labels, node_features, test_size=0.2, random_state=42)

# Create and compile the model
sequential_model = create_sequential_gnn_cnn((224, 224, 3), len(images))
sequential_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
sequential_model.fit([train_node_features, train_data], tf.keras.utils.to_categorical(train_labels, num_classes=10), epochs=20, batch_size=32, validation_data=([test_node_features, test_data], tf.keras.utils.to_categorical(test_labels, num_classes=10)))

# Evaluate the model
def evaluate_model(model, test_data, test_labels):
    test_labels = np.argmax(test_labels, axis=1)  # Convert one-hot encoding to categorical labels
    predictions = model.predict(test_data)
    predicted_labels = np.argmax(predictions, axis=1)
    
    acc = accuracy_score(test_labels, predicted_labels)
    precision = precision_score(test_labels, predicted_labels, average='weighted')
    recall = recall_score(test_labels, predicted_labels, average='weighted')
    f1 = f1_score(test_labels, predicted_labels, average='weighted')
    
    print(f"Accuracy: {acc:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    
    return acc, precision, recall, f1

# Evaluate and save the model
evaluate_model(sequential_model, [test_node_features, test_data], tf.keras.utils.to_categorical(test_labels, num_classes=10))
sequential_model.save('sequential_gnn_cnn.h5')


Epoch 1/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 12s 354ms/step - accuracy: 0.3440 - loss: 1.8981 - val_accuracy: 0.8014 - val_loss: 0.6286
Epoch 2/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - accuracy: 0.8572 - loss: 0.4873 - val_accuracy: 0.9078 - val_loss: 0.3984
Epoch 3/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.9003 - loss: 0.3168 - val_accuracy: 0.9220 - val_loss: 0.3046
Epoch 4/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.9179 - loss: 0.2372 - val_accuracy: 0.9362 - val_loss: 0.2587
Epoch 5/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.9495 - loss: 0.1774 - val_accuracy: 0.9362 - val_loss: 0.2230
Epoch 6/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.9541 - loss: 0.1383 - val_accuracy: 0.9433 - val_loss: 0.2104
Epoch 7/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.9733 - loss: 0.1049 - val_accuracy: 0.9362 - val_loss: 0.2002
Epoch 8/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.9807 - loss: 0.1033 - val_accuracy: 0.9574 -